In [ ]:
!pip install -r requirements.txt


In [ ]:
import cv2
import face_recognition
import sqlite3
from datetime import datetime

# الاتصال بقاعدة البيانات أو إنشاؤها إذا لم تكن موجودة
conn = sqlite3.connect('people.db')
cursor = conn.cursor()

# إنشاء جدول إذا لم يكن موجودًا
cursor.execute('''
CREATE TABLE IF NOT EXISTS detected_people (
    person_id INTEGER PRIMARY KEY AUTOINCREMENT,
    location TEXT,
    time_stamp TEXT
)''')

# تشغيل الكاميرا
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    rgb_frame = frame[:, :, ::-1]  # تحويل BGR إلى RGB
    
    # التعرف على الوجوه
    face_locations = face_recognition.face_locations(rgb_frame)

    for (top, right, bottom, left) in face_locations:
        cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)

        # إدخال البيانات في قاعدة البيانات
        location = f"Top: {top}, Left: {left}, Bottom: {bottom}, Right: {right}"
        time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        cursor.execute('''INSERT INTO detected_people (location, time_stamp)
                          VALUES (?, ?)''', (location, time_stamp))
        conn.commit()

    cv2.imshow('Camera', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# إغلاق الكاميرا وقاعدة البيانات
cap.release()
conn.close()
cv2.destroyAllWindows()


In [ ]:
import pandas as pd
import sqlite3

# الاتصال بقاعدة البيانات
conn = sqlite3.connect('people.db')

# قراءة البيانات باستخدام Pandas
df = pd.read_sql_query("SELECT * FROM detected_people", conn)

# عرض البيانات
print(df)

# إغلاق الاتصال بقاعدة البيانات
conn.close()

# تحليل بسيط: عدد الأشخاص الذين تم اكتشافهم
print("عدد الأشخاص المكتشفين:", len(df))

# تحليل: عرض الأشخاص المكتشفين في فترة زمنية معينة
start_time = '2023-09-25 12:00:00'
end_time = '2023-09-25 12:30:00'
mask = (df['time_stamp'] >= start_time) & (df['time_stamp'] <= end_time)
print(df[mask])
